<a href="https://colab.research.google.com/github/roque-alfaro/taller-eiv-2026/blob/main/3_Ejercicio_Cubo_multidimensional_(R).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio: creación de un cubo
En este ejercicio vamos a armar una tabla de datos multidimensional para el análisis de las brechas de consultas de especialidad médica (CNE). Para ello haremos lo siguiente:

1. Generar un modelo estrella con las métricas y dimensiones de análisis

2. Generar tablas fact y dimensiones

3. Exportar a excel

 # Configurar área de trabajo

In [1]:
#borrar objetos previos
rm(list=ls())
#borrar figuras (si hay)
while(!dev.cur())dev.off()
cat("\014")

# Marcados con '#' es código que se computa. Queda como comentario

# para leer planillas y exportar .csvs
#if(!require("readr")) install.packages("readr", quiet=TRUE)
# para leer planillas excel
#if(!require("readxl")) install.packages("readxl", quiet=TRUE)
# para manipulación de datos
#if(!require("dplyr")) install.packages("dplyr", repos = "https://cran.rstudio.com/", quiet = TRUE)
library(dplyr)

# para archivos .parquet
if(!require("nanoparquet")) {
  install.packages("nanoparquet",
    dependencies = FALSE,
    repos = "https://packagemanager.posit.co/cran/latest",
    quiet=T)
}
# para escribir excels
if(!require("writexl")) {
  install.packages("writexl",
    dependencies = FALSE,
    repos = "https://packagemanager.posit.co/cran/latest",
    quiet=T)
}

# definir carpetas de trabajo
datos_normalizados <- 'https://github.com/rlagosb/taller_eiv/raw/refs/heads/main/datos_normalizados/'
datos_cubos <- './content/cubos/'

# crear carpetas
if (!dir.exists(datos_cubos)){dir.create(datos_cubos, recursive = TRUE)}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: nanoparquet

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘nanoparquet’”
Loading required package: writexl

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘writexl’”


In [2]:
# cargar datos
# --- Centros ---
dest_cen <- tempfile(fileext = ".parquet")
download.file(
    paste0(datos_normalizados, "Centros.parquet"),
    dest_cen,
    mode  = "wb",
    quiet = TRUE
)
centros <- nanoparquet::read_parquet(dest_cen)
# --- Especialidades ---
dest_esp <- tempfile(fileext = ".parquet")
download.file(
    paste0(datos_normalizados, "Especialidades.parquet"),
    dest_esp,
    mode  = "wb",
    quiet = TRUE
)
especialidades <- nanoparquet::read_parquet(dest_esp)
# --- Demanda ---
dest_dem <- tempfile(fileext = ".parquet")
download.file(
    paste0(datos_normalizados, "Demanda.parquet"),
    dest_dem,
    mode  = "wb",
    quiet = TRUE
)
demanda <- nanoparquet::read_parquet(dest_dem)
# --- Producción ---
dest_prod <- tempfile(fileext = ".parquet")
download.file(
    paste0(datos_normalizados, "Produccion.parquet"),
    dest_prod,
    mode  = "wb",
    quiet = TRUE
)
produccion <- nanoparquet::read_parquet(dest_prod)

# Crear tabla FACT
Construiremos una tabla con los campos:

- Centro_cod: código del centro de salud
- Especialidad_cod: código de la especialidad médica (prestación)
- Mes: mes del año 2021
- CNE_solicitadas: solicitudes de interconsultas recibidas por establecimiento, especialidad y mes
- CNE_producidas: consultas nuevas de especialidad producidas por establecimientos, especialidad y mes



In [3]:
# Examinemos los campos que tienen las tablas de demanda y producción
# Con el cruce de estas tablas podemos obtener las métricas fundamentales para el análisis
#Tres primeros valores de la base demanda
demanda %>% head(3)

cat("\n\n") #saltos de línea

#Tres primeros valores de la base producción
produccion %>% head(3)

,Especialidad,Centro_destino,Año,Mes,Interconsultas,__index_level_0__
,<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>
1,07-053,114101,2021,1,822,0
2,07-053,114101,2021,3,748,1
3,07-044,114101,2021,1,673,2


,Año,Mes,Centro,Nuevas,Controles,Codigo
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,2021,1,114101,33,192,07-047
2,2021,1,114101,239,325,07-030
3,2021,1,114101,8,82,07-039


In [4]:
# 1. Seleccionar y renombrar columnas de la tabla demanda
dda <- demanda %>%
  dplyr::select(Centro_destino, Especialidad, Mes, Interconsultas) %>%  # Seleccionar columnas relevantes
  dplyr::rename(
    Centro_cod = Centro_destino,       # Renombrar 'Centro_destino' a 'Centro_cod'
    Especialidad_cod = Especialidad,   # Renombrar 'Especialidad' a 'Especialidad_cod'
    CNE_solicitadas = Interconsultas    # Renombrar 'Interconsultas' a 'CNE_solicitadas'
  )

# 2. Seleccionar y renombrar columnas de la tabla produccion
prod <- produccion %>%
  dplyr::select(Centro, Codigo, Mes, Nuevas) %>%  # Seleccionar columnas relevantes
  dplyr::rename(
    Centro_cod = Centro,              # Renombrar 'Centro' a 'Centro_cod'
    Especialidad_cod = Codigo,        # Renombrar 'Codigo' a 'Especialidad_cod'
    CNE_producidas = Nuevas            # Renombrar 'Nuevas' a 'CNE_producidas'
  )

# 3. Realizar una unión completa (outer join) entre demanda y produccion
fact <- dda %>%
  dplyr::full_join(prod,
                 by = c("Centro_cod",
                 "Especialidad_cod",
                 "Mes"))  # Unir por las columnas especificadas

# 4. Reemplazar valores NA por 0 en las columnas 'CNE_solicitadas' y 'CNE_producidas'
# Utilizamos la función 'coalesce' para reemplazar NAs con 0
fact <- fact %>%
  dplyr::mutate(
    CNE_solicitadas = coalesce(CNE_solicitadas, 0),   # Reemplazar NA en 'CNE_solicitadas' por 0
    CNE_producidas = coalesce(CNE_producidas, 0)    # Reemplazar NA en 'CNE_producidas' por 0
  )

# 5. Convertir las columnas 'CNE_solicitadas' y 'CNE_producidas' a enteros
fact <- fact %>%
  dplyr::mutate(
    CNE_solicitadas = as.integer(CNE_solicitadas),   # Convertir a entero
    CNE_producidas = as.integer(CNE_producidas)      # Convertir a entero
  )

# Mostrar la estructura de la tabla resultante
str(fact)

Classes ‘tbl’ and 'data.frame':	2080 obs. of  5 variables:
 $ Centro_cod      : num  114101 114101 114101 114101 114101 ...
 $ Especialidad_cod: chr  "07-053" "07-053" "07-044" "07-053" ...
 $ Mes             : num  1 3 1 2 9 5 2 3 8 6 ...
 $ CNE_solicitadas : int  822 748 673 655 654 631 629 625 623 600 ...
 $ CNE_producidas  : int  640 668 578 542 615 898 601 768 681 595 ...


## 🏁 Discusión
1. ¿Cuáles son las métricas que registra esta tabla?
1. ¿Cuáles son los hechos que registra esta tabla? ¿Cuántos exiten?
2. ¿Cuál es el nivel de detalle con que se describen los hechos (granularidad)?
1. ¿A partir de esta tabla es posible consultar los hechos a nivel diario? ¿Trimestral? ¿Anual?

# Dimensión Periodo
Aunque nuestro análisis sólo considera un año, podríamos querer analizar los datos a nivel trimestral o anual. Construiremos una tabla con los campos

- Mes: llave primaria
- Trimestre
- Año

**⚠** Si trabajáramos con múltiples años necesitaríamos definir otra llave primaria para esta dimensión

In [5]:
# Creamos un dataframe vacío para agregar los campos
dim_periodo <- cbind.data.frame(
  Mes =         1:12,               # Números del 1 al 12 para los meses
  Año =         2021,               # Año constante 2021
  Trimestre =   rep(1:4, each = 3)  # Asignar trimestre
) %>%
  dplyr::arrange(Mes)  # Ordenar por Mes para mantener el orden cronológico

dim_periodo

Mes,Año,Trimestre
<int>,<dbl>,<int>
1,2021,1
2,2021,1
3,2021,1
4,2021,2
5,2021,2
6,2021,2
7,2021,3
8,2021,3
9,2021,3


# Dimensión Especialidad
Crearemos una tabla con los siguientes campos:

- Especialidad_cod: código de la especialidad (llave primaria)
- Especialidad: nombre de la especialidad
- Alto_riesgo: especialidad con mayor mortalidad para tiempos de espera extensos según Martínez et al ([2019](https://doi.org/10.1186/s12889-019-6526-6))
- REM: si se incluye en reportes estadísticos mensuales


In [6]:
# Examinemos la tabla especialidades para identificar los campos existentes y los que faltan
set.seed(123)#para reproducibilidad
especialidades[sample(nrow(especialidades),5),] #tomamos 5 observaciones al azar

,Codigo,Nombre,Nombre_prog,Codigo_prog,Codigo_rem
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,07-031,MEDICINA INTENSIVA ADULTO,MEDICINA INTENSIVA ADULTO,7030999,NA
2,07-051,REUMATOLOGÍA,REUMATOLOGÍA,7021230,7021230
3,07-014,ENDOCRINOLOGÍA ADULTO,ENDOCRINOLOGÍA ADULTO,7020600,7020600
4,07-042,NEUROLOGÍA PEDIÁTRICA,NEUROLOGÍA PEDIÁTRICA,7022134,7022134
5,07-050,RADIOTERAPIA ONCOLÓGICA,RADIOTERAPIA ONCOLÓGICA,7030502,7030500


In [7]:
# Generemos la dimensión especialidad con una función especial

# 1. Seleccionar y renombrar columnas de la tabla especialidades
esp <- especialidades %>%
  dplyr::select(Codigo, Nombre, Codigo_rem) %>%  # Seleccionar columnas relevantes
  dplyr::rename(
    Especialidad_cod = Codigo,        # Renombrar 'Codigo' a 'Especialidad_cod'
    Especialidad = Nombre             # Renombrar 'Nombre' a 'Especialidad'
  )

# 2. Generar el campo Especialidad_REM
esp <- esp %>%
  # Asignar 'Si' si Codigo_rem no es NA, de lo contrario 'No'
  dplyr::mutate(
    Especialidad_REM = ifelse(!is.na(Codigo_rem), "Si", "No")
  ) %>%
  dplyr::select(-Codigo_rem)  # Eliminar la columna 'Codigo_rem'

 # Identificamos las especialidades de alto riesgo: CARDIOLOGIA,
 #CIRUGIA CARDIOVASCULAR, GASTROENTEROLOGIA ADULTO,
 #NEFROLOGIA ADULTO, NEUROLOGIA ADULTO, UROLOGIA
  alto_riesgo <- c('07-003', '07-006', '07-018', '07-037', '07-041', '07-054')  # Lista de códigos de alto riesgo

  dim_especialidad <- esp %>%
    # Asignar 'Si' si el código está en alto_riesgo,
    #de lo contrario 'No'
    dplyr::mutate(
      Riesgo_mortalidad = ifelse(Especialidad_cod %in% alto_riesgo, "Si", "No")
    )

# Veamos el resultado
dplyr::glimpse(dim_especialidad)
set.seed(123)#para reproducibilidad

#tomamos 5 observaciones al azar
especialidades[sample(nrow(especialidades),5),]

Rows: 68
Columns: 4
$ Especialidad_cod  <chr> "07-001", "07-002", "07-003", "07-004", "07-005", "0…
$ Especialidad      <chr> "ANATOMÍA PATOLÓGICA", "ANESTESIOLOGÍA", "CARDIOLOGÍ…
$ Especialidad_REM  <chr> "No", "Si", "Si", "Si", "Si", "Si", "Si", "Si", "Si"…
$ Riesgo_mortalidad <chr> "No", "No", "Si", "No", "No", "Si", "No", "No", "No"…


,Codigo,Nombre,Nombre_prog,Codigo_prog,Codigo_rem
,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,07-031,MEDICINA INTENSIVA ADULTO,MEDICINA INTENSIVA ADULTO,7030999,NA
2,07-051,REUMATOLOGÍA,REUMATOLOGÍA,7021230,7021230
3,07-014,ENDOCRINOLOGÍA ADULTO,ENDOCRINOLOGÍA ADULTO,7020600,7020600
4,07-042,NEUROLOGÍA PEDIÁTRICA,NEUROLOGÍA PEDIÁTRICA,7022134,7022134
5,07-050,RADIOTERAPIA ONCOLÓGICA,RADIOTERAPIA ONCOLÓGICA,7030502,7030500


## 🏁 Discusión
1. ¿Qué diferencia a esta tabla de la tabla normalizada *Especialidades*?
2. ¿Cuál es el propósito de cada una?

# 🍎 Desafío: Crear Dimensión Establecimiento
Cree un dataframe `dim_centro` con los siguientes campos:

- Centro_cod: código DEIS del centro (llave primaria)
- Centro: nombre del centro de salud
- Complejidad: nivel de complejidad del centro
- Servicio: Servicio de Salud del cual depende

In [8]:
dim_centro <- centros %>%
  dplyr::select(Codigo, Nombre, Dependencia, `Nivel de Complejidad`) %>%  # Seleccionar columnas relevantes. Ojo que para columnas que contienen espacios o parten con caracteres especiales, dplyr requirere el tilde inverso para llamarlos y manipularlos
  dplyr::rename(
    Centro_cod    = Codigo,
    Centro        = Nombre,
    Complejidad   = "Nivel de Complejidad",
    Servicio      = Dependencia
  )
#Miramos el resultado
dim_centro

Centro_cod,Centro,Servicio,Complejidad
<dbl>,<chr>,<chr>,<chr>
126704,Hospital Comunitario Cristina Calderón de Puerto Williams,Servicio de Salud Magallanes,Baja Complejidad
126204,Hospital Naval (Puerto Williams),SEREMI De Magallanes y la Antártica Chilena,Mediana Complejidad
126412,Posta de Salud Rural Cameron,Servicio de Salud Magallanes,Baja Complejidad
126414,Posta de Salud Rural Agua Fresca,Servicio de Salud Magallanes,Baja Complejidad
126102,Hospital Dr. Marco Antonio Chamorro ( Porvenir),Servicio de Salud Magallanes,Baja Complejidad
201364,UNO SALUD DENTAL LOS ÁNGELES 2,SEREMI Del Biobío,Mediana Complejidad
200311,Centro Comunitario de Salud Familiar Dr. Juan Damianovic,Servicio de Salud Magallanes,Baja Complejidad
200710,Complejo Miraflores (Salud Mental),Servicio de Salud Magallanes,Mediana Complejidad
126606,COSAM Punta Arenas,Servicio de Salud Magallanes,Mediana Complejidad


In [9]:
# 🍎 Dimensión Centro / Establecimiento
dim_centro <- centros %>%
  dplyr::select(Codigo, Nombre, Dependencia, `Nivel de Complejidad`) %>%
  dplyr::rename(
    Centro_cod  = Codigo,
    Centro      = Nombre,
    Complejidad = `Nivel de Complejidad`,
    Servicio    = Dependencia
  ) %>%
  dplyr::distinct(Centro_cod, .keep_all = TRUE) %>%   # 1 fila por centro
  dplyr::arrange(Centro_cod)

# Revisar
dplyr::glimpse(dim_centro)


Rows: 4,749
Columns: 4
$ Centro_cod  <dbl> 101010, 101011, 101012, 101030, 101090, 101100, 101203, 10…
$ Centro      <chr> "Actividades gestionadas por la Dirección del Servicio par…
$ Servicio    <chr> "Servicio de Salud Arica y Parinacota", "Servicio de Salud…
$ Complejidad <chr> "Baja Complejidad", "Baja Complejidad", "No Aplica", "Baja…


In [10]:
writexl::write_xlsx(
  dim_centro,
  path = file.path(datos_cubos, "Dim_Centro.xlsx")
)


# Cubo
Generamos una tabla multidimensional uniendo la tabla Fact con las dimensiones utilizando las llaves.

In [11]:
cubo <- fact %>%
  dplyr::left_join(dim_especialidad, by = "Especialidad_cod") %>%  # Unir con dim_especialidad en 'Especialidad_cod'
  dplyr::left_join(dim_centro, by = "Centro_cod") %>%              # Unir con dim_centro en 'Centro_cod'
  dplyr::left_join(dim_periodo, by = "Mes")                        # Unir con dim_periodo en 'Mes'

set.seed(2125)
cubo %>%
  sample_n(5, replace = TRUE)

Centro_cod,Especialidad_cod,Mes,CNE_solicitadas,CNE_producidas,Especialidad,Especialidad_REM,Riesgo_mortalidad,Centro,Servicio,Complejidad,Año,Trimestre
<dbl>,<chr>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>
114101,07-013,4,0,25,DIABETOLOGÍA,Si,No,"Complejo Hospitalario Dr. Sótero del Río (Santiago, Puente Alto)",Servicio de Salud Metropolitano Sur Oriente,Alta Complejidad,2021,2
114105,07-014,12,49,40,ENDOCRINOLOGÍA ADULTO,Si,No,Hospital Clínico Metropolitano La Florida Dra. Eloísa Díaz Insunza,Servicio de Salud Metropolitano Sur Oriente,Alta Complejidad,2021,4
114101,07-048,4,45,27,PSIQUIATRÍA ADULTO,Si,No,"Complejo Hospitalario Dr. Sótero del Río (Santiago, Puente Alto)",Servicio de Salud Metropolitano Sur Oriente,Alta Complejidad,2021,2
200282,07-051,11,2,0,REUMATOLOGÍA,Si,No,Centro de Referencia de Salud Hospital Provincia Cordillera,Servicio de Salud Metropolitano Sur Oriente,Mediana Complejidad,2021,4
114105,07-049,6,42,16,PSIQUIATRÍA PEDIÁTRICA Y DE LA ADOLESCENCIA,Si,No,Hospital Clínico Metropolitano La Florida Dra. Eloísa Díaz Insunza,Servicio de Salud Metropolitano Sur Oriente,Alta Complejidad,2021,2


In [12]:
# Exportamos a la carpeta con los cubos
 writexl::write_xlsx(
    cubo,
    paste0(datos_cubos, "Cubo_brechas_CNE.xlsx")
)

## 🏁 Discusión
1. ¿Qué campos tienen redundancias? (¿Cuál forma normal no se cumple?)
3. ¿Cuál es el propósito de introducir esta redundancia?
4. ¿Existe el riesgo que se produzan inconsistencias en los nombres de los centros o especialidades?
1. ¿Qué cambios tendría que hacer a esta tabla si tuviera:
  - Datos del 2021 al 2024?
  - Datos de más establecimientos?